### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Psoriatic_Arthritis/GSE57386'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Analysis
# 1. Check if gene expression data is available
is_gene_available = True  # Based on the output dataset information

# 2. Variable Availability and Data Type Conversion
sample_characteristics = {
    0: ['Sex: F', 'Sex: M', 'response: .', 'response: responder', 'response: nonresponder'],
    1: ['age: 51', 'age: 28', 'age: 46', 'age: 57', 'age: 61', 'age: 35', 'age: 19', 'age: 67', 'age: 38', 'age: 55', 'age: 39', 'age: 44', 'age: 52', 'Sex: F', 'Sex: M', 'age: 54', 'age: 40', 'age: 64', 'age: 23', 'age: 60',
        'age: 32', 'age: 24', 'age: 62', 'age: 42', 'age: 36', 'age: 56', 'age: 47', 'age: 50', 'age: 66', 'age: 58'],
    2: ['sample type: biopsy', 'age: 40', 'age: 54', 'age: 36', 'age: 23', 'age: 42', 'age: 24', 'age: 62', 'age: 46', 'age: 56', 'age: 32', 'age: 47', 'age: 60', 'age: 44', 'age: 64', 'age: 19', 'age: 61', 'age: 57',
        'age: 35', 'age: 53', 'age: 59', 'age: 28', 'age: 39', 'age: 38', 'age: 52', 'age: 70', 'age: 58', 'age: 68', 'age: 67', 'age: 31'],
    3: ['tissue: knee', 'tissue: Lesional skin', 'tissue: nonlesional skin', 'tissue: synfluid', 'cell type: primary cell', 'cell subtype: CD14-'],
    4: ['disease status: diseased', 'cell subtype: CD14+', 'disease status: normal'],
    5: ['disease: Rheumatoid Arthritis', 'disease: Psoriasis', 'disease: Psoriatic Arthritis', 'disease: normal', 'disease: diseased', 'disease: Healthy Control'],
    6: ['time point: wk0', 'time point: wk2', 'time point: wk10', 'disease: Health Control', 'disease: Psoriasis', 'disease: Psoriatic Arthritis', 'disease: Rheumatoid Arthritis'],
    7: [nan, 'time point: wk0', 'time point: wk2', 'time point: wk10']
}

# Determine availability and keys
# 'Psoriatic_Arthritis' availability and key
for key, values in sample_characteristics.items():
    if 'disease: Psoriatic Arthritis' in values:
        trait_row = key
        break

# 'age' availability and key
age_keys = [1, 2]
for key in age_keys:
    if any('age:' in v for v in sample_characteristics[key]):
        age_row = key
        break

# 'gender' availability and key
for key, values in sample_characteristics.items():
    if 'Sex: F' in values or 'Sex: M' in values:
        gender_row = key
        break

# Data type conversion functions
def convert_trait(value: str) -> int:
    if 'disease: Psoriatic Arthritis' in value:
        return 1
    elif 'disease:' in value:
        return 0
    return None

def convert_age(value: str) -> float:
    try:
        _, age_val = value.split(': ')
        return float(age_val)
    except ValueError:
        return None

def convert_gender(value: str) -> int:
    if 'Sex: F' in value:
        return 0
    elif 'Sex: M' in value:
        return 1
    return None

# Save Metadata
save_cohort_info('GSE57386', './preprocessed/Psoriatic_Arthritis/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Psoriatic_Arthritis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Psoriatic_Arthritis/trait_data/GSE57386.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
